# BallHolder RL Pipeline\n
\n
Run order: analyze dataset -> create splits (+ optional HF upload) -> analyze splits -> train -> benchmark.

In [ ]:
# Environment setup (run once)\n
!pip install -q datasets pillow numpy scipy wandb python-dotenv\n
\n
# Ensure your keys exist in .env or pass --env-file to each command\n
!cat .env

## 1) Analyze original dataset

In [ ]:
# Default\n
!python analyze_ballholder_dataset.py \
  --dataset-name maxs-m87/Ball-Holder \
  --split train \
  --streaming \
  --out-json outputs/ballholder_dataset_stats_raw.json\n
\n
# Recommended (larger sample + shuffle)\n
!python analyze_ballholder_dataset.py \
  --dataset-name maxs-m87/Ball-Holder \
  --split train \
  --streaming --shuffle --buffer-size 5000 --seed 42 \
  --max-samples 100000 \
  --out-json outputs/ballholder_dataset_stats_raw.json

## 2) Create splits (and optionally upload to HF)

In [ ]:
# Default local split build\n
!python create_ballholder_splits.py \
  --dataset maxs-m87/Ball-Holder \
  --split train \
  --val-fraction 0.10 \
  --holdout-count 1000 \
  --empty-fraction 0.25\n
\n
# Recommended: also push to hub (edit repo name)\n
!python create_ballholder_splits.py \
  --dataset maxs-m87/Ball-Holder \
  --split train \
  --val-fraction 0.10 \
  --holdout-count 1000 \
  --empty-fraction 0.25 \
  --push-to-hub maxs-m87/Ball-Holder-splits-v1 \
  --hub-val-split validation \
  --hub-post-val-split test

## 3) Analyze splits to validate positive/negative balance

In [ ]:
# Analyze locally saved train/val/post_val splits\n
!python analyze_ballholder_dataset.py \
  --dataset-path outputs/maxs-m87_Ball-Holder_splits \
  --all-splits \
  --out-json outputs/ballholder_dataset_stats_splits.json

## 4) Train (with W&B metrics)

In [ ]:
# Default train command\n
!python train_ballholder.py \
  --dataset-path outputs/maxs-m87_Ball-Holder_splits \
  --split train \
  --val-split val \
  --num-steps 1000 \
  --batch-size 8 \
  --group-size 4 \
  --max-objects 1 \
  --best-metric eval_miou \
  --eval-every 25 \
  --save-every 25 \
  --wandb-project moondream-ballholder-rl\n
\n
# Recommended stronger run\n
!python train_ballholder.py \
  --dataset-path outputs/maxs-m87_Ball-Holder_splits \
  --split train \
  --val-split val \
  --num-steps 1500 \
  --batch-size 8 \
  --group-size 8 \
  --lr 0.002 \
  --max-objects 1 \
  --augment-prob 0.9 \
  --empty-keep-prob 0.5 \
  --off-policy \
  --best-metric eval_miou \
  --eval-every 25 \
  --save-every 25 \
  --eval-max-samples 2000 \
  --wandb-project moondream-ballholder-rl

## 5) Benchmark on unseen split (`post_val`)

In [ ]:

FINETUNE_ID = ""\n
!python benchmark_ballholder.py \
  --finetune-id  "01KHQ1H5BBP3G74PKFTER45F5Q" \
  --checkpoint-step 200 \
  --dataset-name maxs-m87/Ball-Holder-splits-v1 \
  --split test \
  --batch-size 16 --max-workers 16 \
  --max-objects 1 \
  --iou-threshold 0.4 \
  --save-viz \
  --viz-limit 20 \
  --out-json outputs/staging_01KHQ1H5BBP3G74PKFTER45F5Q.json
  --skip-baseline
  

  python benchmark_ballholder.py \
  --finetune-id "01KHQ1JF28VPWXTRCF7NA1DXJE" \
  --checkpoint-step 180 \
  --dataset-name maxs-m87/Ball-Holder-splits-v1 \
  --split test \
  --batch-size 16 \
  --max-workers 16 \
  --max-objects 1 \
  --iou-threshold 0.4 \
  --save-viz \
  --viz-limit 20 \
  --out-json outputs/staging_01KHQ1JF28VPWXTRCF7NA1DXJE.json \
  --skip-baseline \
  --api-base https://api-staging.moondream.ai/v1